In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

In [43]:
import re
import pickle
from pathlib import Path

import gurobipy
from gurobipy import GRB
import dgl
import numpy as np
import torch

from src.data import get_model, get_soc, load_instance

In [54]:
instances = sorted(list(Path('../data/raw').glob('97_9*.jl')))

instance = load_instance(instances[0])

J = instance['jobs'][0]
T = instance['tamanho'][0]
uso_p = instance['uso_p']
recurso_p = instance['recurso_p']

jobs = list(range(J))
model = get_model(jobs, instance, coupling=True)

with open('../97_9_opts.pkl', 'rb') as f:
    opts = pickle.load(f)

x_opt = opts[instances[0].name]['sol']
x_opt.shape

(1746,)

In [69]:
model_vars = np.core.defchararray.array([v.getAttr(GRB.Attr.VarName) for v in model.getVars()])
model_vars = model_vars[model_vars.find('soc') == -1]  # drop soc vars
sol = x_opt[model_vars.find('phi') == -1]
sol_vars = model_vars[model_vars.find('phi') == -1]

sol_idx = [re.fullmatch(r"x\((\d+),(\d+)\)", s_v).groups() for s_v in sol_vars]
sol_idx = np.array(list(map(lambda jt: list(map(int, jt)), sol_idx)))

solucao = np.zeros_like(sol).reshape((J, T))
for sol_jt, (j, t) in zip(sol, sol_idx):
    solucao[j,t] = sol_jt

In [81]:
def benders_subproblem(instance, solucao, solve=True, verbose=True):
    subproblem = gurobipy.Model()
    if verbose:
        subproblem.Params.LogToConsole = 1

    lmbd0 = {}
    lmbd1 = {}
    lmbd2 = {}
    lmbd3 = {}
    lmbd4 = {}
    lmbd5 = {}
    lmbd6 = {}
    lmbd7 = {}

    J = instance['jobs'][0]
    T = instance['tamanho'][0]
    uso_p = instance['uso_p']
    recurso_p = instance['recurso_p']

    soc_inicial = 0.7
    limite_inferior = 0.0
    ef = 0.9
    v_bat = 3.6
    q = 5
    bat_usage = 5

    for t in range(T):
        lmbd1[t] = subproblem.addVar(name="lmbd1(%s)" % t, lb=0, vtype=GRB.CONTINUOUS) # alpha 1 
        lmbd2[t] = subproblem.addVar(name="lmbd2(%s)" % t, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS) # b 2 
        lmbd3[t] = subproblem.addVar(name="lmbd3(%s)" % t, lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS) # b 3
        lmbd4[t] = subproblem.addVar(name="lmbd4(%s)" % t, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS) # soc - i 4
        lmbd5[t] = subproblem.addVar(name="lmbd5(%s)" % t, lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS) # soc - i 5
        lmbd6[t] = subproblem.addVar(name="lmbd6(%s)" % t, lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS) # limite inferior 6
        lmbd7[t] = subproblem.addVar(name="lmbd7(%s)" % t, lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS) # limite inferior 6

    for t in range(T):
        subproblem.addConstr((bat_usage * v_bat)*lmbd1[t] + lmbd7[t] >= 0)
        subproblem.addConstr(lmbd2[t] - lmbd3[t]/v_bat == 0)
        subproblem.addConstr(lmbd3[t] -  (ef / q)*(1/60) * lmbd4[t] == 0)

    for t in range(T-1):
            subproblem.addConstr(lmbd4[t] - lmbd4[t+1] + (-lmbd5[t] + lmbd6[t]) == 0)

    subproblem.addConstr(lmbd4[T-1] + (-lmbd5[T-1] + lmbd6[T-1]) == 0)
    subproblem.setParam('InfUnbdInfo', 1)
    subproblem.Params.LogToConsole = 0
    subproblem.update()

    obj = 0

    for t in range(T):
        lhs = float(sum(uso_p[j] * solucao[j][t] for j in range(J)))
        obj += lmbd1[t] * ((recurso_p[t] + bat_usage * v_bat) - lhs) + lmbd2[t] * (recurso_p[t] - lhs) - 0.0*lmbd5[t] + lmbd6[t] + lmbd7[t]
    obj += lmbd4[0] * 0.7
    subproblem.setObjective(obj, GRB.MINIMIZE)
    subproblem.update()

    if solve:
        subproblem.optimize()

    return subproblem

benders_subproblem(instance, solucao).status

Set parameter InfUnbdInfo to value 1


5

In [73]:
subproblem.status

5

Let's try with the original implementation of the problem.

In [77]:
J = instance['jobs'][0]
T = instance['tamanho'][0]
uso_p = instance['uso_p']
recurso_p = instance['recurso_p']

priority = instance['priority'] # prioridade de cada tarefa
uso_p = instance['uso_p'] # recurso utilizado por cada tarefa
min_statup = instance['min_statup'] # tempo mínimo de vezes que uma tarefa pode iniciar
max_statup = instance['max_statup'] # tempo máximo de vezes que uma tarefa pode iniciar
min_cpu_time = instance['min_cpu_time'] # tempo mínimo de unidades de tempo que uma tarefa pode consumir em sequência
max_cpu_time = instance['max_cpu_time'] # tempo máximo de unidades de tempo que uma tarefa pode consumir em sequência
min_periodo_job = instance['min_periodo_job'] # tempo mínimo que uma tarefa deve esperar para se repetir
max_periodo_job = instance['max_periodo_job'] # tempo máximo que uma tarefa pode esperar para se repetir
win_min = instance['win_min']
win_max = instance['win_max']

model = gurobipy.Model()
model.Params.LogToConsole = 1

x = {}
alpha = {}
soc = {}
i = {}
b = {}
phi = {}
for j in range(J):
    for t in range(T):
        x[j,t] = model.addVar(name="x(%s,%s)" % (j, t), lb=0, ub=1, vtype=gurobipy.GRB.BINARY)
        phi[j,t] = model.addVar(vtype=gurobipy.GRB.CONTINUOUS, name="phi(%s,%s)" % (j, t),)


for t in range(T):
    alpha[t] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="alpha(%s)" % t)
    soc[t] = model.addVar(vtype=GRB.CONTINUOUS, name="soc(%s)" % t)
    i[t] = model.addVar(lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="i(%s)" % t)
    b[t] = model.addVar(lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="b(%s)" % t)

soc_inicial = 0.7
limite_inferior = 0.0
ef = 0.9 
v_bat = 3.6 
q = 5
bat_usage = 5

# set objective
model.setObjective(sum(priority[j] * x[j,t] for j in range(J) for t in range(T)), gurobipy.GRB.MAXIMIZE)

# phi defines startups of jobs
for t in range(T):
  for j in range(J):
    if t == 0:
        model.addConstr(phi[j,t] >= x[j,t] - 0)
    else:
        model.addConstr(phi[j,t] >= x[j,t] - x[j,t - 1])

    model.addConstr(phi[j,t] <= x[j,t])

    if t == 0:
        model.addConstr(phi[j,t] <= 2 - x[j,t] - 0)
    else:
        model.addConstr(phi[j,t] <= 2 - x[j,t] - x[j,t - 1])

# minimum and maximum number of startups of a job
for j in range(J):
  model.addConstr(sum(phi[j,t] for t in range(T)) >= min_statup[j])
  model.addConstr(sum(phi[j,t] for t in range(T)) <= max_statup[j])

  ###############################
  # precisa ajustar

  # execution window
  model.addConstr(sum(x[j,t] for t in range(win_min[j])) == 0)
  model.addConstr(sum(x[j,t] for t in range(win_max[j], T)) == 0)

for j in range(J):
  # minimum period between jobs
  for t in range(0, T - min_periodo_job[j] + 1):
    model.addConstr(sum(phi[j,t_] for t_ in range(t, t + min_periodo_job[j])) <= 1)

  # periodo máximo entre jobs
  for t in range(0, T - max_periodo_job[j] + 1):
    model.addConstr(sum(phi[j,t_] for t_ in range(t, t + max_periodo_job[j])) >= 1)

  # min_cpu_time das jobs
  for t in range(0, T - min_cpu_time[j] + 1):
    model.addConstr(sum(x[j,t_] for t_ in range(t, t + min_cpu_time[j])) >= min_cpu_time[j] * phi[j,t])

  # max_cpu_time das jobs
  for t in range(0, T - max_cpu_time[j]):
      model.addConstr(sum(x[j,t_] for t_ in range(t, t + max_cpu_time[j] + 1)) <= max_cpu_time[j])

  # min_cpu_time no final do periodo
  for t in range(T - min_cpu_time[j] + 1, T):
      model.addConstr(sum(x[j,t_] for t_ in range(t, T)) >= (T - t) * phi[j,t])

################################
# Add power constraints
for t in range(T):
  model.addConstr(sum(uso_p[j] * x[j,t] for j in range(J)) <= recurso_p[t] + bat_usage * v_bat)# * (1 - alpha[t]))
  
################################
# Bateria
################################

for t in range(T):
      model.addConstr(sum(uso_p[j] * x[j,t] for j in range(J)) + b[t] == recurso_p[t])


# Define the i_t, SoC_t constraints in Gurobi
for t in range(T):
    # P = V * I 
    model.addConstr(b[t] / v_bat >= i[t])

    if t == 0:
        # SoC(1) = SoC(0) + p_carga[1]/60
        model.addConstr(soc[t] == soc_inicial + (ef / q) * (i[t] / 60))
    else:
        # SoC(t) = SoC(t-1) + (ef / Q) * I(t)
        model.addConstr(soc[t] == soc[t - 1] + (ef / q) * (i[t] / 60))

    # Set the lower and upper limits on SoC
    model.addConstr(limite_inferior <= soc[t])
    model.addConstr(soc[t] <= 1)

model.update()

model.optimize()

model.status

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 5649 rows, 2134 columns and 62539 nonzeros
Model fingerprint: 0x98c669cc
Variable types: 1164 continuous, 970 integer (970 binary)
Coefficient statistics:
  Matrix range     [3e-03, 1e+01]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e-01, 3e+01]
Presolve removed 1469 rows and 506 columns
Presolve time: 0.07s
Presolved: 4180 rows, 1628 columns, 44605 nonzeros
Variable types: 848 continuous, 780 integer (780 binary)
Found heuristic solution: objective 503.0000000

Root relaxation: objective 1.247879e+03, 1902 iterations, 0.05 seconds (0.10 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     

2

In [87]:
model.NumVars

2134

In [88]:
model.getA().shape

(5649, 2134)

In [86]:
solucao = np.zeros((J,T))
for j in range(J):
    for t in range(T):
        solucao[j,t] = x[j,t].X

benders_subproblem(instance, solucao).status

Set parameter InfUnbdInfo to value 1


2